# Get Duration

In [ ]:
# @title
!pip install ffmpeg

In [ ]:
# @title
# Get duration for list of url that accept Cross-Origin
import subprocess
import requests

# List of MP3 URLs
mp3_urls = [
    "https://archive.org/download/DPTK2_NgheAudio/25.mp3",
    "https://archive.org/download/DPTK2_NgheAudio/26.mp3",
    "https://archive.org/download/DPTK2_NgheAudio/27.mp3"
]

# Optional: headers to mimic a browser
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Referer": "https://archive.org"
}

def get_duration_from_url(url):
    try:
        # Stream the file to ffprobe without saving
        with requests.get(url, headers=HEADERS, stream=True) as r:
            r.raise_for_status()
            #ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 https://example.com/song.mp3

            # Run ffprobe
            cmd = [
                "ffprobe",
                "-v", "error",
                "-show_entries", "format=duration",
                "-of", "default=noprint_wrappers=1:nokey=1",
                url
                #"-"  # read from stdin
            ]
            # Pipe the streamed content to ffprobe
            process = subprocess.run(cmd, input=r.raw.read(), capture_output=True)
            duration = process.stdout.decode().strip()
            return float(duration)
    except Exception as e:
        print(f"Failed to get duration for {url}: {e}")
        return None

def format_seconds(seconds: int) -> str:
    seconds = int(seconds)
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    secs = seconds % 60

    if hours > 0:
        return f"{hours}:{minutes:02}:{secs:02}"
    else:
        return f"{minutes}:{secs:02}"

durs = []
# Process all URLs
for item in mp3_urls:
    duration = get_duration_from_url(item)
    if duration:
        durs.append({"url": item, "dur": format_seconds(duration)});
        print(f"{item} → {format_seconds(duration)}")
print(list(durs))

''' durs = map(lambda t: {"url": t, "dur": format_seconds(get_duration_from_url(t))}, mp3_urls)
print(list(durs)) '''

# Process all URLs
''' for url in mp3_urls:
    duration = get_duration_from_url(url)
    if duration:

        print(f"{url} → {format_seconds(duration)}") '''


# reformat for radiotruyen.me

DATA

In [ ]:
# @title
# DATA
data = [
  {
    "title": "Vũ Động Càn Khôn - Tập 1",
    "mp3": "https://files.radiotruyen.me/4817--vdck/dwYVhXb%2B7MnYDuLewl4Auw==.mp3",
    "dur": 3842.99102
  },
  {
    "title": "Vũ Động Càn Khôn - Tập 2",
    "mp3": "https://files.radiotruyen.me/4817--vdck/HMMc4cthJadQybm%2BJ6EgvQ==.mp3",
    "dur": 3896.672653
  },
  {
    "title": "Vũ Động Càn Khôn - Tập 3",
    "mp3": "https://files.radiotruyen.me/4817--vdck/suaGs%2BDawhVGdd%2Btp2OB%2Bw==.mp3",
    "dur": 3660.682449
  }
]

FUNCTION

In [ ]:
# @title
# Convert data to JSON format that can used in my Book Audio apps


def format_seconds(seconds: int) -> str:
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    secs = seconds % 60

    if hours > 0:
        return f"{hours}:{minutes:02}:{secs:02}"
    else:
        return f"{minutes}:{secs:02}"

b = list( map(lambda t: {"tit": t["title"].replace("Vũ Động Càn Khôn - ",""), "url": t["mp3"],"dur": format_seconds(int(t["dur"]))} , data))

#c="\n".join(list(map(lambda t: str(t), b)))
c="\n".join(list(map(repr, b)))


def copy_to_clipboard1(text):
    from IPython.display import Javascript, display
    display(Javascript(f'navigator.clipboard.writeText(`{text}`)'))
def copy_to_clipboard(text):
    from IPython.display import Javascript, HTML
    display(HTML(f"""
    <input type="text" value="{text}" id="copyInput" style="position:absolute; left:-1000px;">
    <button onclick="var copyText = document.getElementById('copyInput'); copyText.select(); document.execCommand('copy'); alert('Copied to clipboard!');">
        Copy to Clipboard
    </button>
    """))
#copy_to_clipboard("[" + "\n".join(list(map(repr, b))) + "]")
#display(c)

''' import pyperclip
pyperclip.copy(c) '''
print(str(b))

# Reformat the titles for Xich Tam Tuan Thien

In [ ]:
# @title
!pip install pyperclip

In [ ]:
# @title
# reformat the titles for Xich Tam Tuan Thien
import re#, pyperclip
from pprint import pprint

A = [
    {
        "tit": "[TẬP 912] XÍCH TÂM TUẦN THIÊN - Audio Tiên hiệp 2025 | MC Tiến Phong",
        "dur": "44:31",
        "img": "urv1F8aGeWE"
    },
    {
        "tit": "[TẬP 915] XÍCH TÂM TUẦN THIÊN : Phong Tuyết Nhân Gian - Audio Tiên hiệp 2025 | MC Tiến Phong",
        "dur": "46:16",
        "img": "VsoPIM4N54c"
    },
    {
        "tit": "[TẬP 916] XÍCH TÂM TUẦN THIÊN : Thân Tráng Khung Lư - Audio Tiên hiệp 2025 | MC Tiến Phong",
        "dur": "52:31",
        "img": "xw5RZJVlEGo"
    },
    {
        "tit": "[TẬP 917] XÍCH TÂM TUẦN THIÊN : Thiên Vấn - Audio Tiên hiệp 2025 | MC Tiến Phong",
        "dur": "41:03",
        "img": "V10DD4LlrUA"
    }
]

new_A = []

for item in A:
    tit = item.get("tit", "")
    match = re.match(r"\[TẬP\s*(\d+)\].*?:\s*(.+?)\s*-", tit)

    if match:
        ep_num, main_title = match.groups()
        formatted_title = f"Tập {ep_num} – {main_title.strip()}".title()
        # Rebuild dict with 'tit' first
        new_item = {"tit": formatted_title}
        for k, v in item.items():
            if k != "tit":
                new_item[k] = v
        new_A.append(new_item)
    else:
        # If no main title, keep other properties only
        new_item = {k: v for k, v in item.items() if k != "tit"}
        new_A.append(new_item)

#pprint(new_A)
print("[" + ",\n".join(list(map(repr, new_A))) + "]")

# Copy to clipboard
# pyperclip.copy(str(new_A))